# FLAX

Permet de construire des réseaux de neurone avec des classes, comme en torch, ou en tensorflow (Class-API de keras)

## Construire un modèle

### Un layer

In [2]:
import jax
from flax import linen as nn
from jax import random
import jax.numpy as jnp
import jax.random as jr

In [3]:
# Redéfinir une couche Dense simple en utilisant setup()
class LinearLayer(nn.Module):
    in_dim: int
    dim: int

    def setup(self):
        # Déclaration et initialisation des poids (kernel) dans setup()
        self.kernel = self.param(
            'kernel',
            jax.nn.initializers.lecun_normal(),
            (self.in_dim, self.dim),
            jnp.float32,
        )

        self.bias = self.param(
            'bias',
            jax.nn.initializers.zeros,
            (self.dim,),
            jnp.float32,
        )

    def __call__(self, x):
        y = jnp.dot(x, self.kernel)+self.bias
        return y

In [4]:
def test_LinearLayer():
    inp_dim=3
    linearLayer=LinearLayer(inp_dim,5)
    dummy_inp=jnp.zeros([1,inp_dim])

    params=linearLayer.init(jr.key(0),dummy_inp)
    print(jax.tree.map(lambda tens:tens.shape,params))

    inp=jnp.ones([1,inp_dim])
    out=linearLayer.apply(params,inp)
    print(out.shape)
test_LinearLayer()

Explication sur la ligne:

    self.kernel = self.param(
                'kernel',                           #Un nom
                jax.nn.initializers.lecun_normal(), #Une fonction (rkey,shape,dtype)->un tenseur
                (self.in_features, self.features),  #la shape
                jnp.float32,                        #le dtype
            )

la méthode `param` de `nn.Module` sera lancée par `nn.Module.init()`. Elle crée un tenseur via la fonctionn passée en paramtre, et l'enregistre dans le pytree `params` qui sera renvoyé par `nn.Module.init()`.

In [5]:
#testons jax.nn.initializers.lecun_normal()
#oui, c'est une fonction!
kernel=jax.nn.initializers.lecun_normal()(jr.key(0),(3,5),jnp.float32)
kernel

### Emboiter des layers

In [7]:
class MLP(nn.Module):
    n_layer: int
    inp_dim: int
    hidden_dim: int
    out_dim: int

    def setup(self):
        self.in_layers=LinearLayer(self.inp_dim,self.hidden_dim)
        hidden_layers=[]
        for _ in range(self.n_layer-1):
            hidden_layers.append(LinearLayer(self.hidden_dim,self.hidden_dim))
        #Les attributs doivent être immuables,
        #donc on transforme la liste en tuple
        self.hidden_layers=tuple(hidden_layers)

        self.final_layer=LinearLayer(self.hidden_dim,self.out_dim)

    def __call__(self, x):
        # Utiliser les sous-modules définis dans setup() pour le forward pass
        x = jnp.tanh(self.in_layers(x))
        for layer in self.hidden_layers:
            x=jnp.tanh(layer(x))
        x = self.final_layer(x)
        return x

In [8]:
def test_MLP():
    n_layer= 5
    inp_dim= 2
    hidden_dim= 16
    out_dim=3
    model=MLP(n_layer,inp_dim,hidden_dim,out_dim)

    batch_size = 1
    dummy_inp = jnp.zeros([batch_size, inp_dim])
    params=model.init(jr.key(0),dummy_inp)
    print(jax.tree.map(lambda tens:tens.shape,params))

    inp = jnp.zeros([batch_size, inp_dim])
    out=model.apply(params,inp)
    print(out.shape)
test_MLP()

### Utiliser un layer prédéfinit

In [ ]:
class MLP2(nn.Module):
    n_layer: int
    inp_dim: int
    hidden_dim: int
    out_dim: int

    def setup(self):
        self.in_layers=nn.Dense(self.hidden_dim)
        hidden_layers=[]
        for _ in range(self.n_layer-1):
            hidden_layers.append(nn.Dense(self.hidden_dim))
        #Les attributs doivent être immuables,
        #donc on transforme la liste en tuple
        self.hidden_layers=tuple(hidden_layers)
        self.final_layer=nn.Dense(self.out_dim)

    def __call__(self, x):
        # Utiliser les sous-modules définis dans setup() pour le forward pass
        x = jnp.tanh(self.in_layers(x))
        for layer in self.hidden_layers:
            x=jnp.tanh(layer(x))
        x = self.final_layer(x)
        return x

Remarquez que `nn.Dense` n'a pas besoin de la dim d'entrée !!!

## Utilisation de @nn.compact

La transformation `@nn.compact` permet de ne pas écrire la méthode `setup`, et permet de ne pas demmander à l'utilisateur la dimension des inputs.




### Le layer

In [ ]:
class LinearLayer2(nn.Module):
    dim: int

    @nn.compact
    def __call__(self, x):
        #on lit la dimension des inputs sur `x`
        in_dim=x.shape[-1]

        # Déclaration et initialisation des poids (kernel) dans setup()
        kernel = self.param(
            'kernel',
            jax.nn.initializers.lecun_normal(),
            (in_dim, self.dim), # Correct shape using in_dim
            jnp.float32,
        )

        bias = self.param(
            'bias',
            jax.nn.initializers.zeros,
            (self.dim,),
            jnp.float32,
        )

        y = jnp.dot(x, kernel)+bias
        return y

### Le modèle

Et idem pour le MLP, on peut tout mettre dans la méthode `__call__`:

In [ ]:
class MLP3(nn.Module):
    n_layer: int
    hidden_dim: int
    out_dim: int

    @nn.compact
    def __call__(self, x):
        in_layers=nn.Dense(self.hidden_dim)
        hidden_layers=[]
        for _ in range(self.n_layer-1):
            hidden_layers.append(nn.Dense(self.hidden_dim))

        hidden_layers=tuple(hidden_layers)
        final_layer=nn.Dense(self.out_dim)

        # Utiliser les sous-modules définis dans setup() pour le forward pass
        x = jnp.tanh(in_layers(x))
        for layer in hidden_layers:
            x=jnp.tanh(layer(x))
        x = final_layer(x)
        return x

In [ ]:
def test_MLP3():
    n_layer= 5
    inp_dim= 2
    hidden_dim= 16
    out_dim=3
    model=MLP3(n_layer,hidden_dim,out_dim)

    batch_size = 1
    dummy_inp = jnp.zeros([batch_size, inp_dim])
    params=model.init(jr.key(0),dummy_inp)
    print(jax.tree.map(lambda tens:tens.shape,params))

    inp = jnp.zeros([batch_size, inp_dim])
    out=model.apply(params,inp)
    print(out.shape)

test_MLP3()

Voilà, c'est plus concis et l'utilisateur n'a pas besoin d'indiquer la dimension d'entrée. Elle est lu au premier appel de dans la méthode `__call__`, et c'est à ce moment que l'on initialise les paramètres. Cela s'appelle l'initialisation tardive. Elle est aussi utilisé dans la lib `stax` (incluse dans JAX) et dans tenserflow.keras. Par contre torch ne l'utilise pas.




### Explications détaillées

(par Gemini, relu et adapté par le prof)

Dans les modules Flax, la méthode `__call__` définit le passage avant (forward pass) des données à travers le module. Cependant, lorsqu'elle est combinée avec le décorateur `@compact` et la méthode `self.param()`, elle gère également l'initialisation des paramètres.

1.  **`@compact` Decorator**: Ce décorateur modifie le comportement de la méthode `__call__`. Il permet à `__call__` de servir à la fois à définir l'architecture du module et à initialiser les paramètres la première fois qu'elle est exécutée dans le contexte de `model.init()`. Cela simplifie la définition des modules en évitant d'avoir une méthode `setup` séparée.

2.  **`self.param(name, initializer, *args)`**: C'est la méthode utilisée pour déclarer et gérer les paramètres (variables qui font partie de l'état entraînable du modèle, comme les poids et les biais).
    *   **Lors de l'initialisation (`model.init(key, dummy_input)`)**: Lorsque `self.param()` est appelée pour la première fois pour un paramètre donné (identifié par `nom_du_module/name`), Flax vérifie s'il existe déjà. S'il n'existe pas, Flax utilise l'`initializer` fourni (par exemple, `jax.nn.initializers.lecun_normal()` ou `jax.nn.initializers.zeros`) pour créer la valeur du paramètre avec la forme spécifiée par `*args` et l'ajoute à la structure de variables retournée par `init`.
    *   **Lors de l'inférence (`model.apply(variables, useful_input)`)**: Lorsque `self.param()` est appelée dans le contexte de `apply`, Flax recherche le paramètre par `nom_du_module/name` dans la structure `variables` qui lui a été passée. Il récupère simplement la valeur existante du paramètre et l'utilise dans le calcul. Il ne réinitialise pas le paramètre.

En résumé, la méthode `__call__` décorée avec `@compact` utilise `self.param()` pour déclarer les besoins en paramètres du module. Flax gère ensuite l'initialisation de ces paramètres la première fois que le module est "runnable" (généralement lors de l'appel à `init`) et réutilise ces paramètres lors des appels suivants à `apply`. Cela sépare l'état du modèle de la définition de son architecture.

## Défit prog: Fourier Embeding


### C'est quoi

C'est un layer qui transforme l'input `x` en un plongement constitué des

    sin(dot_product)
    cos(dot_product)

où

    dot_product = x@B

où `B` est une matrice alétoire, avec des coefs gaussien ayant une scale fixée par l'utilisateur.  

Ainsi, si `x` est de dimension `inp_dim`. alors `B` doit être de shape `(inp_dim,out_dim)` où `out_dim` est choisi par l'utilisateur.


Ce plongement de l'input dans une série de cos/sin permet de faire de la régression sur des fonctions très oscillante. On vera un exemple dans un autre TP.


Il y a deux écoles: ceux qui veule que `B` soit entrainable, et ceux qui ne le souhaite pas. Dans mes test perso, rendre `B` entrainable n'améliore pas pas l'apprentissage.  



On doit va coder un layer avec un paramètre `learnable_frequencies=True/False` pour indiquer si `B` est entrainable ou non.  

### Implémentation non satisfaisante

Une IA m'a proposé cette solution, mais je n'en suis pas satisfait. Annalysez pourquoi.

In [22]:
class FourierEmbsIA(nn.Module):
    scale: float
    learnable_frequencies: bool
    num_frequencies: int = 256

    @nn.compact
    def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
        input_dims = x.shape[-1]  # Récupérer la dimension de l'entrée ici
        b_shape = (input_dims, self.num_frequencies)

        if self.learnable_frequencies:
            B_matrix = self.param('frequencies_B', nn.initializers.normal(stddev=1.0), b_shape, jnp.float32) * self.scale
        else:
            B_variable = self.variable('constants', 'frequencies_B', lambda: jax.random.normal(self.make_rng('params'), b_shape) * self.scale)
            B_matrix = B_variable.value  # Accéder à la valeur du tenseur JAX de la variable

        # Calculer le produit scalaire x * B
        dot_product = jnp.dot(x, B_matrix)*2*jnp.pi

        # Appliquer sin et cos
        fourier_features_sin = jnp.sin(dot_product)
        fourier_features_cos = jnp.cos(dot_product)

        fourier_features = jnp.concatenate([fourier_features_sin,fourier_features_cos], axis=-1)
        return fourier_features

def test2(learnable_frequencies):
    inp_dim = 3
    scale=1.
    fourierEmbs = FourierEmbsIA(scale, learnable_frequencies)
    inp = jnp.zeros([1, inp_dim])
    params = fourierEmbs.init(jr.key(0), inp)
    print(jax.tree.map(lambda x:x.shape,params))

    out = fourierEmbs.apply(params, inp)
    print(out.shape)


In [23]:
test2(learnable_frequencies=True)


In [24]:
test2(learnable_frequencies=False)

## Notre implémentation

Complétez la classe ci-dessus.

In [25]:
class FourierEmbs(nn.Module):
    inp_dim: int
    scale:float
    learnable_frequencies:bool
    num_frequencies:int = 256

    def setup(self):
        #float32 ou float64 en fonction de la config globale de JAX
        dtype = jnp.array(1.).dtype
        ...


In [26]:
def test(learnable_frequencies):
    inp_dim = 3
    scale=1.
    fourierEmbs = FourierEmbs(inp_dim,scale, learnable_frequencies)
    inp = jnp.zeros([1, inp_dim])
    params = fourierEmbs.init(jr.key(0), inp)
    print(jax.tree.map(lambda x:x.shape,params))

    out = fourierEmbs.apply(params, inp)
    print(out.shape)

In [27]:
test(learnable_frequencies=True)

In [27]:
#--- To keep following outputs, do not run this cell! ---

{'params': {'frequencies_B': (3, 256)}}
(1, 512)


In [28]:
test(learnable_frequencies=False)

In [28]:
#--- To keep following outputs, do not run this cell! ---

{}
(1, 512)
